In [1]:
!git clone https://github.com/doaa1adel/UERC-Ear-Recognition-Challenge

Cloning into 'UERC-Ear-Recognition-Challenge'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 24 (delta 3), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (24/24), done.


In [2]:
cd /content/UERC-Ear-Recognition-Challenge

/content/UERC-Ear-Recognition-Challenge


In [23]:
import os,shutil
import cv2,keras
import numpy as np
from imgaug import augmenters as iaa
import skimage.io as io
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.optimizers import Adam

In [ ]:
def create_dataset(path,num_classes,data_splitno):
    data_split=[]
    subjects = os.listdir(path) # getting each subject folders (no. of classes=166)
    for subject in subjects[:num_classes]:     ## selcting only 150 classes
        os.makedirs(os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/train1',subject))  ## creating new folder for train and test with 60/40 split from each subjects
        os.makedirs(os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/test1',subject))
        subject_images = os.listdir(os.path.join(path,subject))
        for image in subject_images:
            if(image.endswith('.png')):
                data_split.append(image)
                train_no = int(data_splitno*len(data_split))
        for i in range(0,len(data_split)):
        
            if(i<=train_no-1):
                source_train=os.path.join(path,subject,data_split[i])
                destination_train=os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/train1',subject)
                shutil.copy(source_train,destination_train)
        
            else:
                source_test=os.path.join(path,subject,data_split[i])
                destination_test=os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/test1',subject)
                shutil.copy(source_test,destination_test)
                
        data_split=[] 

In [ ]:
def data_augmentation(image,path):
    flip =iaa.Sequential([iaa.Fliplr(0.5)]).augment_images(image)
    io.imsave(path+'flip.png',flip)
    
    crop = iaa.Sequential([iaa.Crop(percent=(0,0.01))]).augment_images(image)
    io.imsave(path+'crop.png',crop)
    
    GB = iaa.Sequential([iaa.GaussianBlur(sigma=(0,3.0))]).augment_images(image)
    io.imsave(path+'GB.png',GB)
    
    GN = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=(0.0,0.2))]).augment_images(image)
    io.imsave(path+'GN.png',GN)
    
    CN = iaa.Sequential([iaa.ContrastNormalization(0.5,per_channel=0.5)]).augment_images(image)
    io.imsave(path+'CN.png',CN)
    
    BR = iaa.Sequential([iaa.Multiply((0.8,1.2))]).augment_images(image)
    io.imsave(path+'BR.png',BR)
    
    scale= iaa.Sequential([iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8,1.2)})]).augment_images(image)
    io.imsave(path+'scale.png',scale)
    
    rotate = iaa.Sequential([iaa.Affine(rotate=(-45,45))]).augment_images(image)
    io.imsave(path+'rotate.png',rotate)
    

In [ ]:
def get_augmented_data(path):
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(100,100))
                im_path=os.path.join(os.path.splitext(image_path)[0])
                data_augmentation(image,im_path)

In [5]:
def data(path):
    images=[]
    label=[]
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(224,224))  #100,100  224,224
                images.append(image)
                label.append(subject)
    
    images=np.array(images)
    label=np.array(label)
    return images,label
    

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pwd  #to check the current path

/content/UERC-Ear-Recognition-Challenge


In [ ]:
#path = 'Dataset/Train Dataset' #the original command
path= '/content/gdrive/My Drive/Colab Notebooks/Train Dataset'
num_classes=150
data_splitno=0.6
create_dataset(path,num_classes,data_splitno)

# Data Augmentation
train_path = '/content/gdrive/My Drive/Colab Notebooks/Data/train1'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Data/test1'
get_augmented_data(train_path)
get_augmented_data(test_path)



/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py:553: UserWarning: You provided a numpy array of shape (100, 100, 3) as input to augment_images(), which was interpreted as (N, H, W). The last dimension however has value 1 or 3, which indicates that you provided a single image with shape (H, W, C) instead. If that is the case, you should use augment_image(image) or augment_images([image]), otherwise you will not get the expected augmentations.
  "you will not get the expected augmentations." % (images_copy.shape,))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: /content/gdrive/My Drive/Colab Notebooks/Data/train1/164/53CN.png is a low contrast image
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: /content/gdrive/My Drive/Colab Notebooks/Data/test1/154/278CN.png is a low contrast image
  from ipykernel import kernelapp as app


In [6]:
#Getting train and test data(80/20)
train_path = '/content/gdrive/My Drive/Colab Notebooks/Data/train1'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Data/test1'
train_images,train_label= data(train_path)
test_images,test_label= data(test_path)
x_train,x_valid,y_train,y_valid=train_test_split(train_images,train_label,test_size=0.2,random_state=42)
x_test=test_images
y_test=test_label

# convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train,1000)
y_valid= keras.utils.to_categorical(y_valid,1000)
y_test= keras.utils.to_categorical(y_test,1000)
print('x_train=',x_train.shape)
print('y_train=',y_train.shape)
print('x_test=',x_test.shape)
print('y_test=',y_test.shape)

x_train= (9900, 224, 224, 3)
y_train= (9900, 1000)
x_test= (8361, 224, 224, 3)
y_test= (8361, 1000)


In [7]:
model = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=x_train[0].shape, 
                pooling=None, classes=1000)   #weights='imagenet' #weights=None


102858752/102853048 [==============================] - 1s 0us/step


In [8]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [9]:
# for resnet
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=70,epochs=2,verbose=1,validation_data=(x_valid, y_valid))



Train on 9900 samples, validate on 2475 samples
Epoch 1/2
9900/9900 [==============================] - 863s 87ms/step - loss: 3.5524 - accuracy: 0.2907 - val_loss: 5.6461 - val_accuracy: 0.0731
Epoch 2/2
9900/9900 [==============================] - 882s 89ms/step - loss: 0.7840 - accuracy: 0.8205 - val_loss: 1.1910 - val_accuracy: 0.6909


In [17]:
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
print('Test accuracy:',test_acc)
print('train accuracy:', train_acc)


Test accuracy: 0.24099987745285034
train accuracy: 0.7427272796630859
